In [ ]:
!pip install -q --no-index --find-links /kaggle/input/ultralytics ultralytics

In [ ]:
%%writefile convert_dcm.py 

import os
import pydicom
from PIL import Image
import numpy as np
from multiprocessing import Pool, cpu_count

import sklearn.metrics
import torch
import cv2
import numpy as np 
import pandas as pd 
from tqdm.auto import tqdm
IMG_DIR = '/images'
def read_dcm(src_path):
    dicom_data = pydicom.dcmread(src_path)
    image = dicom_data.pixel_array
    image = (image - image.min()) / (image.max() - image.min() +1e-6) * 255
    return image

def convert_dcm_to_jpg(file_path):
    try:
        # Read the DICOM file
        image_array = read_dcm(file_path)
        
        # Define the output path
        relative_path = os.path.relpath(file_path, start=input_directory)
        output_path = os.path.join(output_directory, relative_path)
        output_path = output_path.replace('.dcm', '.jpg')
                
        # Create the output directory if it doesn't exist
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # Save the image as a JPEG file
        cv2.imwrite(output_path, image_array)
        
        return output_path
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

def process_files(dcm_files):
    with Pool(cpu_count()) as pool:
        # Wrap pool.map with tqdm to show the progress bar
        list(tqdm(pool.imap(convert_dcm_to_jpg, dcm_files), total=len(dcm_files)))

def get_dcm_files(directory):
    dcm_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.dcm'):
                dcm_files.append(os.path.join(root, file))
    return dcm_files    


input_directory = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images'

output_directory = IMG_DIR

# Get all .dcm files in the input directory
dcm_files = get_dcm_files(input_directory)

# Process the files using multiprocessing
process_files(dcm_files)

print(f"Conversion completed. Images saved to {output_directory}")

In [ ]:
!python convert_dcm.py

In [ ]:
%%writefile infer.py 

import sys
model_dir = sys.argv[1]
img_size = int(sys.argv[2])
fold_index = int(sys.argv[3])
save_file = sys.argv[4]
#weight = sys.argv[5]
print(model_dir, img_size, fold_index)

import os
import pydicom
from PIL import Image
import numpy as np
from multiprocessing import Pool, cpu_count

import sklearn.metrics
import torch
import cv2
import numpy as np 
import pandas as pd 
from tqdm.auto import tqdm

EVAL = 0 
IMG_DIR = '/images'
FOLD = int(fold_index)
SAMPLE = 0 
SEVERITIES = ['Normal/Mild', 'Moderate', 'Severe']
LEVELS = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']

SCS_WEIGHTS = [f'{model_dir}/scs_fold{FOLD}_{img_size}_normal.pt']
SS_WEIGHTS = [f'{model_dir}/ss_fold{FOLD}_{img_size}_normal.pt']
NFN_WEIGHTS = [f'{model_dir}/nfn_fold{FOLD}_{img_size}_normal.pt']

SCS_WEIGHTS += [f'{model_dir}/scs_fold{FOLD}_{img_size}_severe.pt']
SS_WEIGHTS += [f'{model_dir}/ss_fold{FOLD}_{img_size}_severe.pt']
NFN_WEIGHTS += [f'{model_dir}/nfn_fold{FOLD}_{img_size}_severe.pt']

print(SCS_WEIGHTS,'\n', SS_WEIGHTS,'\n', NFN_WEIGHTS)

train_val_df = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv')
des = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_series_descriptions.csv')


test_df = os.listdir('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images')
test_df = pd.DataFrame(test_df, columns=['study_id'])
test_df['study_id'] = test_df['study_id'].astype(int)
    
test_df = test_df.merge(des, on=['study_id'])

def gen_label_map(CONDITIONS):
    label2id = {}
    id2label = {}
    i = 0
    for cond in CONDITIONS:
        for level in LEVELS:
            for severity in SEVERITIES:
                cls_ = f"{cond.lower().replace(' ', '_')}_{level}_{severity.lower()}"
                label2id[cls_] = i
                id2label[i] = cls_
                i+=1
    return label2id, id2label
                
scs_label2id, scs_id2label = gen_label_map(['Spinal Canal Stenosis'])
ss_label2id, ss_id2label = gen_label_map(['Left Subarticular Stenosis', 'Right Subarticular Stenosis'])
nfn_label2id, nfn_id2label = gen_label_map(['Left Neural Foraminal Narrowing', 'Right Neural Foraminal Narrowing'])

from ultralytics import YOLO

# Load YOLO Model
scs_models = []
for weight in SCS_WEIGHTS:
    scs_models.append(YOLO(weight))

ss_models = []
for weight in SS_WEIGHTS:
    ss_models.append(YOLO(weight))

nfn_models = []
for weight in NFN_WEIGHTS:
    nfn_models.append(YOLO(weight))

all_label_set = train_val_df.iloc[0, 1:].index.tolist()
scs_label_set = all_label_set[:5]
nfn_label_set = all_label_set[5:15]
ss_label_set = all_label_set[15:]

settings = [
    ( 'Sagittal T2/STIR', scs_models, scs_id2label, scs_label_set, 0.01),
    ( 'Axial T2', ss_models, ss_id2label, ss_label_set, 0.02),
    ( 'Sagittal T1', nfn_models, nfn_id2label, nfn_label_set, 0.02)
]

from collections import defaultdict

pred_rows = []

for modality, models, id2label, label_set, thresh in settings:
    mod_df = test_df[test_df.series_description == modality]
    
    if SAMPLE:
        mod_df = mod_df.sample(20, random_state=610)
    
    # for each study, at each level and condition, get the maximum probability score
    for study_id, group in tqdm(mod_df.groupby('study_id')):
        predictions = defaultdict(list)
        for i, row in group.iterrows():
            # predict on all images from all the series
            series_dir = os.path.join(IMG_DIR, str(row['study_id']), str(row['series_id']))
            for model in models:
                results = model(series_dir, imgsz=img_size, conf=thresh, verbose=False, augment=False, )
                for res in results:
                    for pred_class, conf in zip(res.boxes.cls, res.boxes.conf):
                        pred_class = pred_class.item()
                        conf = conf.item()
                        _class = id2label[pred_class]
                        predictions[_class].append(conf)
        
        # aggregate the result on images to obtain study-level prediction
        for condition in label_set:
            res_dict = {'row_id': f'{study_id}_{condition}' }

            score_vec = []
            for severity in SEVERITIES:
                if severity=='Severe':
                    scaler = 2.0
                elif severity=='Moderate':
                    scaler = 1.2
                else:
                    scaler = 0.85
                severity = severity.lower()
                key = f'{condition}_{severity}'
                if len(predictions[key]) > 0:
                    score = np.max(predictions[key])
                else:
                    score = thresh
                    
                if score>0.2:
                    score = score*scaler
                    
                score_vec.append(score)
                
            # normalize score to sum to 1
            score_vec = torch.tensor(score_vec)
            score_vec = score_vec / score_vec.sum()

            for idx, severity in enumerate(SEVERITIES):
                res_dict[severity.replace('/', '_').lower()] = score_vec[idx].item()

            pred_rows.append(res_dict)


pred_df = pd.DataFrame(pred_rows)
pred_df

pred_df.to_csv(save_file, index=False)




In [ ]:
#/kaggle/input/rsna2024-yolos4/nfn_fold0_384_normal.pt

In [ ]:
!python infer.py /kaggle/input/rsna2024-yolos4 384 0 pred7.csv

In [ ]:
!python infer.py /kaggle/input/rsna2024-yolos4 512 0 pred8.csv

In [ ]:
!python infer.py /kaggle/input/rsna2024-yolos4 768 0 pred9.csv

In [ ]:
!python infer.py /kaggle/input/rsna2024-yolos4 384 1 pred10.csv

In [ ]:
!python infer.py /kaggle/input/rsna2024-yolos4 512 1 pred11.csv

In [ ]:
!python infer.py /kaggle/input/rsna2024-yolos4 768 1 pred12.csv

In [ ]:
!python infer.py /kaggle/input/rsna2024-yolos3/yolov8m_last 384 0 pred13.csv

In [ ]:
!python infer.py /kaggle/input/rsna2024-yolos3/yolov8m_last 512 0 pred14.csv

In [ ]:
!python infer.py /kaggle/input/rsna2024-yolos3/yolov8m_last 384 1 pred15.csv

In [ ]:
!python infer.py /kaggle/input/rsna2024-yolos3/yolov8m_last 512 1 pred16.csv

In [ ]:
#%%writefile yolo_ensemble.py
import glob
import pandas as pd
subs = glob.glob('/kaggle/working/pred*.csv')
preds = []
for f in subs:
    df = pd.read_csv(f)
    preds.append(df)
preds = pd.concat(preds, ignore_index=True)
preds = preds.groupby(['row_id']).agg({'normal_mild':"mean", 'moderate':"mean", 'severe':"mean"}).sort_index().reset_index()
preds.to_csv('submission.csv', index=False)